In [4]:
def query_res(word, ans):
    def get_t(word: str, ans: str, i: int) -> int:
        cnt: int = 0
        for j in range(5):
            if ans[j] == word[i] and ans[j] != word[j]:
                cnt += 1
        return cnt
    def get_s(word: str, ans: str, i: int) -> int:
        cnt: int = 0
        for j in range(i+1):
            if word[j] == word[i] and ans[j] != word[j]:
                cnt += 1
        return cnt
    res = [2, 2, 2, 2, 2]
    for i, c in enumerate(word):
        if ans[i] != c:
            t = get_t(word, ans, i)
            s = get_s(word, ans, i)
            res[i] = int(t >= s)
    return res

In [5]:
import numpy as np
import tqdm

save = []
def query_res_all(ans: np.ndarray, words: np.ndarray) -> np.ndarray:
    N: int = words.shape[0]

    not_greens = (ans != words) # (N, 5)

    matches = (words[:, :, None] == ans[None, None, :]) # (N, 5, 5)
    '''
    matches[x][i][j]
    x번째 단어의 i번째 문자와 답의 j번째 문자가 같은지 여부
    '''
    t = (matches & not_greens[:, None, :]).sum(axis=2) # (N, 5)
    '''
    def get_t(word: str, ans: str, i: int) -> int:
        cnt: int = 0
        for j in range(5):
            if ans[j] == word[i] and ans[j] != word[j]:
                cnt += 1
        return cnt
    '''

    matches = (words[:, :, None] == words[:, None, :]) # (N, 5, 5)
    '''
    matches[x][i][j]
    x번째 단어의 i번째 문자와 x번째 단어의 j번째 문자가 같은지 여부
    '''
    s = np.cumsum(matches & not_greens[:, None, :], axis=2)[np.arange(N)[:, None], np.arange(5), np.arange(5)] # (N, 5)
    '''
    def get_s(word: str, ans: str, i: int) -> int:
        cnt: int = 0
        for j in range(i+1):
            if word[j] == word[i] and ans[j] != word[j]:
                cnt += 1
        return cnt
    '''
    
    res = np.full_like(words, fill_value=2, dtype=int) # (N, 5)
    res[not_greens] = (t >= s)[not_greens].astype(int)

    # test
    '''
    for i in range(words.shape[0]):
        if query_res(words[i], ans) != list(res[i]):
            print("ans:", ans, "query:", words[i], query_res(words[i], ans), list(res[i]))
            break
    '''

    res = (res * np.array([1, 3, 9, 27, 81])).sum(axis=1) # (N,)
    save.append(res)

    counts = np.zeros(273)
    counts[res] += 1
    return counts
    
words = np.array(list(map(list, open('code\words.txt').read().strip().split('\n')))) # (N, 5)

def calc_entropy(word):
    counts = query_res_all(word, words)
    probs = counts/counts.sum()
    probs_ = probs.copy()
    probs_[probs == 0] = 1
    return -np.sum(probs * np.log2(probs_))

# ent = {word:calc_entropy(word) for word in words}
for word in tqdm.tqdm(words):
    calc_entropy(word)

print(np.array(save).shape)
np.save('all.npy', np.array(save))

  7%|▋         | 859/12972 [00:05<01:23, 144.78it/s]


KeyboardInterrupt: 

`all.npy`에 저장하고 불러와서 엔트로피를 구하자

In [ ]:
import numpy as np
from time import time

start_time = time()

words = open('code\words.txt').read().strip().split('\n') # (N, 5)
res = np.load('all.npy')
print(time() - start_time)

candidates = words
word_to_index = {word: i for i, word in enumerate(words)}
idx = np.array([word_to_index[w] for w in candidates])  # (C,)
responses = res[np.ix_(idx, idx)]  # shape: (C, C)
print(time() - start_time)

entropy = np.zeros(len(candidates))
for j in range(len(candidates)):
    r = responses[:, j]
    _, counts = np.unique(r, return_counts=True)
    probs = counts / counts.sum()
    entropy[j] = -np.sum(probs * np.log2(probs))

print(time() - start_time)

1.0869624614715576
1.7829666137695312
7.1992292404174805


In [21]:
def query_res(word, ans):            
    feedback = np.zeros(5)
    for i in range(5):
        if word[i] == ans[i]:
            feedback[i] = 2

    remaining = {}
    for i, ch in enumerate(ans):
        if feedback[i] != 2:
            remaining[ch] = remaining.get(ch, 0) + 1

    for i in range(5):
        if feedback[i] != 2 and remaining.get(word[i], 0) > 0:
            feedback[i] = 1
            remaining[word[i]] -= 1

    return feedback

print(query_res(word=np.array(list("apian")), ans=np.array(list("fugal"))))

[0. 0. 0. 2. 0.]


In [11]:
words = open('code/words.txt').read().strip().split('\n') # (N, 5)
word_to_index = {word: i for i, word in enumerate(words)}
all_results = np.load('code/all.npy') # (N, N)

print(all_results[word_to_index["fugal"]][word_to_index["apian"]])

54


# use 4-dim array
메모리 이슈로 망함

In [41]:
import numpy as np
from time import time

words = np.array(list(map(list, open('code\words.txt').read().strip().split('\n')[:4000]))) # (N, 5)
N = words.shape[0]

start_time = time()

def calc_entropy(word):
    counts = query_res_all(word, words)
    probs = counts/counts.sum()
    probs_ = probs.copy()
    probs_[probs == 0] = 1
    return -np.sum(probs * np.log2(probs_))

'''
matches[x][y][i][j]
x번째 단어의 i번째 문자와 y번째 단어의 j번째 문자가 같은지 여부
'''
matches = (words[:, None, :, None] == words[None, :, None, :]) # (N, N, 5, 5) 
print(time() - start_time)

not_greens = ~matches[:, :, np.arange(5), np.arange(5)] # (N, N, 5)
print(time() - start_time)

t = (matches & not_greens[:, :, None, :]).sum(axis=3) # (N, N, 5)
print(time() - start_time)

s = np.cumsum((matches[np.arange(N), np.arange(N), :, :][:, None, :, :] & not_greens[:, :, None, :]), axis=3)[:, :, np.arange(5), np.arange(5)] # (N, N, 5)
print(time() - start_time)

res = np.full_like(not_greens, fill_value=2, dtype=int) # (N, N, 5)
res[not_greens] = (t >= s)[not_greens].astype(int)
print(time() - start_time)


# res = (res * np.array([1, 3, 9, 27, 81])).sum(axis=1) # (N,)

# counts = np.zeros(273)
# counts[res] += 1

# check
'''
for i in range(N):
    for j in range(N):
        if query_res(words[j], words[i]) != list(res[j][i]):
            print("ans:", words[i], "query:", words[j], query_res(words[j], words[i]), list(res[j][i]))
            break
'''



2.009331226348877
2.2751193046569824
4.165659189224243
8.021244764328003
8.960752010345459


'\nfor i in range(N):\n    for j in range(N):\n        if query_res(words[j], words[i]) != list(res[j][i]):\n            print("ans:", words[i], "query:", words[j], query_res(words[j], words[i]), list(res[j][i]))\n            break\n'

In [ ]:
# simple version
import numpy as np

words = np.array(list(map(list, open('code\words.txt').read().strip().split('\n')[:4000]))) # (N, 5)
N = words.shape[0]



# Use Query_res2

In [21]:
def query_res2(word, ans):
    dict = {}
    for c in ans:
        if c in dict:
            dict[c] += 1
        else:
            dict[c] = 1

    res = []
    for cw, ca in zip(word, ans):
        if cw == ca:
            res.append(2)
            dict[cw] -= 1
        elif (cw not in dict) or dict[cw] == 0:
            res.append(0)
        else:
            res.append(1)
            dict[cw]-=1
    return res

In [22]:
print(query_res('aalid', 'abacd'))
print(query_res2('aalid', 'abacd'))

[2, 1, 0, 0, 2]
[2, 1, 0, 0, 2]


In [24]:
import time

word_list = open('./code/words.txt').read().strip().split('\n')[:4000]

start_time = time.time()
def calc_entropy2(word):
    li = [tuple(query_res2(word=word,ans=j)) for j in word_list]
    _, counts = np.unique(li, return_counts=True, axis=0)
    probs = counts/counts.sum()
    return -np.sum(probs * np.log2(probs))

ent = {word:calc_entropy2(word) for word in word_list}
guess = max(ent, key=ent.get)

print(f'calculating entropy using query_res2: {time.time() - start_time}, {guess}')



start_time = time.time()
def calc_entropy(word):
    li = [tuple(query_res(word=word,ans=j)) for j in word_list]
    _, counts = np.unique(li, return_counts=True, axis=0)
    probs = counts/counts.sum()
    return -np.sum(probs * np.log2(probs))


ent = {word:calc_entropy(word) for word in word_list}
guess = max(ent, key=ent.get)

print(f'calculating entropy using query_res: {time.time() - start_time}, {guess}')



calculating entropy using query_res2: 84.05509614944458, dares
calculating entropy using query_res: 359.8509097099304, dares
